In [1]:
import pandas as pd
import numpy as np
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv,find_dotenv
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

from operator import itemgetter
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage
from pydantic import BaseModel, Field
from typing import Annotated

In [2]:
load_dotenv(find_dotenv("../.env"))

True

<h3> Creating a Tool </h3>

In [ ]:
@tool
def multiply(firstInt:int, secondInt: int) -> int:
    """
        Multiply two integers togather
    """
    return firstInt*secondInt

@tool
def add(firstInt:int, secondInt: int) -> int:
    """
        Add two integers togather
    """
    return firstInt+secondInt

In [4]:
type(multiply)

langchain_core.tools.structured.StructuredTool

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers togather
{'firstInt': {'title': 'Firstint', 'type': 'integer'}, 'secondInt': {'title': 'Secondint', 'type': 'integer'}}


In [6]:
multiply.invoke(input={"firstInt":6,"secondInt":3})

18

<h3>Chains</h3>

In [10]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini")
llmGemini

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000021AE70E7690>, default_metadata=())

In [11]:
llmWithTools=llmGPT.bind_tools(tools=[multiply,add])
llmWithTools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AF1156010>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021AEE8C7BD0>, root_client=<openai.OpenAI object at 0x0000021AE71019D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AF150FF50>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply two integers togather', 'parameters': {'properties': {'firstInt': {'type': 'integer'}, 'secondInt': {'type': 'integer'}}, 'required': ['firstInt', 'secondInt'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'add', 'description': 'Add two integers togather', 'parameters': {'properties': {'firstInt': {'type': 'integer'}, 'secondInt': {'type': 'integer'}}, 'required': ['firstInt', 'secondInt'], 'type': 'object'}}}]}, config={}, config_f

In [12]:
question="What's 5 plus Forty Two?"
response=llmWithTools.invoke(input=question)

In [16]:
response.tool_calls[0]['name'],response.tool_calls[0]['id'],response.tool_calls[0]['args']

('add', 'call_UIbl6OHY7UfORf1pg0qslp7X', {'firstInt': 5, 'secondInt': 42})

In [17]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UIbl6OHY7UfORf1pg0qslp7X', 'function': {'arguments': '{"firstInt":5,"secondInt":42}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 80, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CyfoPLbaAR5DkQDz1ZuWoXu76hjoX', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--985a758a-b172-42e0-97d9-e7cd8c973587-0', tool_calls=[{'name': 'add', 'args': {'firstInt': 5, 'secondInt': 42}, 'id': 'call_UIbl6OHY7UfORf1pg0qslp7X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 19, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_rea

In [18]:
response.content

''

In [19]:
response.tool_calls

[{'name': 'add',
  'args': {'firstInt': 5, 'secondInt': 42},
  'id': 'call_UIbl6OHY7UfORf1pg0qslp7X',
  'type': 'tool_call'}]

In [20]:
add.invoke(response.tool_calls[0]['args'])

47

In [21]:
(llmWithTools|RunnableLambda(lambda k : k.tool_calls[0]["args"])| add).invoke(input=question)

47

<h3>Agents</h3>

In [22]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

In [26]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [24]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [17]:
prompt.input_variables,prompt.optional_variables

(['agent_scratchpad', 'input'], ['chat_history'])

In [27]:
@tool 
def exponentiate (base:int, exponent:int) -> int:
    """
        Exponentiate the base to the exponent power
    """
    return base**exponent   

In [28]:
tools=[multiply, add, exponentiate]

In [29]:
agent=create_tool_calling_agent(llm=llmGPT, tools=tools, prompt=prompt)

In [30]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [31]:
agentExecutor=AgentExecutor(tools=tools,agent=agent, verbose=True)

In [32]:
agentExecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [33]:
question="Take 3 to the fifth power and Multiply that by the sum of twelve and three, then square the whole result"

In [34]:
agentExecutor.invoke(input={
    "input": question
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'firstInt': 12, 'secondInt': 3}`


15
Invoking: `multiply` with `{'firstInt': 243, 'secondInt': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The final result is \( 13,286,025 \).

> Finished chain.


{'input': 'Take 3 to the fifth power and Multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The final result is \\( 13,286,025 \\).'}

<h3>This is Gold !</h3>

In [83]:
llmWithTools=llmGPT.bind_tools(tools=[exponentiate,add,multiply])
toolTracingDict={'exponentiate':exponentiate,'add':add,'multiply':multiply}
messages=[HumanMessage(content=question)]

while messages:
    if isinstance(messages[-1],HumanMessage) or isinstance(messages[-1],ToolMessage) or isinstance(messages[-1],HumanMessage):
        response=llmWithTools.invoke(input=messages)
        messages.append(response)
    # elif messages[-1].tool_calls and messages[-1].content=="":
    elif isinstance(messages[-1],AIMessage) and messages[-1].content=="":
        for tool_call in messages[-1].tool_calls:
            tool_name=toolTracingDict[tool_call['name']]
            tool_id=tool_call['id']
            tool_args=tool_call['args']
            toolMessage=tool_name.invoke(input=tool_args)
            messages.append(ToolMessage(content=toolMessage,tool_call_id=tool_id))
    elif isinstance(messages[-1],AIMessage) and messages[-1].content:
        break




In [84]:
messages

[HumanMessage(content='Take 3 to the fifth power and Multiply that by the sum of twelve and three, then square the whole result', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5ZRJbc3Pqihir6IsvH7lHA88', 'function': {'arguments': '{"base": 3, "exponent": 5}', 'name': 'exponentiate'}, 'type': 'function'}, {'id': 'call_RD0k9xdsBUSlrtqz3X5fGmyE', 'function': {'arguments': '{"firstInt": 12, "secondInt": 3}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 124, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CyhEvY3pDkLjxE3jy7NMe7GFA83l0', 'finish_reason': 'tool_calls', '

In [85]:
messages[-1].content

'The final result of the calculation is **13,286,025**.'

In [100]:
class FinalOutput(BaseModel):
    value:Annotated[int, Field(ge=0)] | Annotated[str,Field(max_length=50)]

In [101]:
prompt=ChatPromptTemplate.from_template(template="Given a Sentence: {sentence},either present it as a string if it is absolutely required, else just present it as an integer value")

In [102]:
chain=prompt|llmGPT.with_structured_output(schema=FinalOutput)

In [103]:
resp=chain.invoke(input={'sentence':messages[-1].content})

In [105]:
resp.value

13286025